In [1]:
import torch
from endurance.variable import Variable
from endurance.activations import relu



#I use the test functions from https://github.com/karpathy/micrograd/blob/master/test/test_engine.py

def test_sanity_check():

    x = Variable(-4.0)
    z = 2 * x + 2 + x
    q = relu(z) + z * x
    h = relu(z * z)
    y = h + q + q * x
    y.backward()
    xmg, ymg = x, y

    x = torch.Tensor([-4.0]).double()
    x.requires_grad = True
    z = 2 * x + 2 + x
    q = z.relu() + z * x
    h = (z * z).relu()
    y = h + q + q * x
    y.backward()
    xpt, ypt = x, y

    # forward pass went well
    assert ymg.value == ypt.data.item()
    # backward pass went well
    assert xmg.gradient == xpt.grad.item()

def test_more_ops():

    a = Variable(-4.0)
    b = Variable(2.0)
    c = a + b
    d = a * b + b**3
    c += c + 1
    c += 1 + c + (-a)
    d += d * 2 + relu(b + a)
    d += 3 * d + relu(b - a)
    e = c - d
    f = e**2
    g = f / 2.0
    g += 10.0 / f
    g.backward()
    amg, bmg, gmg = a, b, g

    a = torch.Tensor([-4.0]).double()
    b = torch.Tensor([2.0]).double()
    a.requires_grad = True
    b.requires_grad = True
    c = a + b
    d = a * b + b**3
    c = c + c + 1
    c = c + 1 + c + (-a)
    d = d + d * 2 + (b + a).relu()
    d = d + 3 * d + (b - a).relu()
    e = c - d
    f = e**2
    g = f / 2.0
    g = g + 10.0 / f
    g.backward()
    apt, bpt, gpt = a, b, g

    tol = 1e-6
    # forward pass went well
    assert abs(gmg.value - gpt.data.item()) < tol
    # backward pass went well
    assert abs(amg.gradient - apt.grad.item()) < tol
    assert abs(bmg.gradient - bpt.grad.item()) < tol

In [2]:
#all good
test_sanity_check()

In [3]:
#all good
test_more_ops()